Retail Sales Trend Analysis and Customer Insights

1. Introduction
This project aims to analyze sales data to uncover trends, customer behaviors, and store performance insights. The analysis is divided into several key sections, including data cleaning, exploratory analysis, statistical insights, and actionable recommendations.


Data Loading

In [ ]:
import pandas as pd

def load_data(file_path):
    """
    Reads a CSV file and returns a pandas DataFrame.

    Parameters:
    file_path (str): The path to the CSV file.

    Returns:
    pd.DataFrame: The DataFrame containing the data from the CSV file, or None if an error occurs.
    """
    try:
        df = pd.read_csv(file_path)
        return df
    except FileNotFoundError:
        print(f"Error: The file at {file_path} was not found.")
        return None
    except pd.errors.EmptyDataError:
        print("Error: The file is empty.")
        return None
    except pd.errors.ParserError:
        print("Error: The file could not be parsed.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage
# df = load_data('data.csv')

## 2. Data Cleaning
This section focuses on preparing the dataset by:
- Removing duplicates
- Handling missing values
- Formatting date and time columns

Data Cleaning

In [ ]:
def clean_data(df):
    """
    Cleans the input DataFrame by removing duplicates and filling missing numerical values with their median.

    Parameters:
    df (pd.DataFrame): The input DataFrame to be cleaned.

    Returns:
    pd.DataFrame: The cleaned DataFrame.
    """
    # Remove duplicates
    df_cleaned = df.drop_duplicates()

    # Fill missing numerical values with their median
    numerical_features = df_cleaned.select_dtypes(include=['int64', 'float64']).columns
    for feature in numerical_features:
        median_value = df_cleaned[feature].median()
        df_cleaned[feature].fillna(median_value, inplace=True)

    return df_cleaned

# Example usage
# cleaned_df = clean_data(df)

## 3. Exploratory Data Analysis
- **Sales Trend Analysis**: Understanding overall trends in sales over time.
- **Product Performance**: Identifying top-selling and underperforming products.
- **Customer Behavior**: Analyzing purchasing trends by time and day.


Exploratory Data Analysis (EDA)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

def plot_sales_trend(df, date_column, sales_column):
    """
    Generates a line plot of sales trends over time with a rolling mean.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    date_column (str): The name of the column containing date values.
    sales_column (str): The name of the column containing sales values.

    Returns:
    None
    """
    # Remove outliers from the sales column
    df = remove_outliers(df, sales_column)

    # Convert the date column to datetime format
    df[date_column] = pd.to_datetime(df[date_column])

    # Sort the DataFrame by date
    df = df.sort_values(by=date_column)

    # Calculate rolling mean
    df['Rolling_Mean'] = df[sales_column].rolling(window=7).mean()

    # Set the plot style
    sns.set(style="whitegrid")

    # Generate the line plot with scatter and rolling mean
    plt.figure(figsize=(14, 7))
    plt.plot(df[date_column], df[sales_column], marker='o', linestyle='-', color='b', label='Sales')
    plt.plot(df[date_column], df['Rolling_Mean'], color='r', linestyle='--', label='7-Day Rolling Mean')
    plt.scatter(df[date_column], df[sales_column], color='b', s=10)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.title('Sales Trend Over Time with Rolling Mean', fontsize=16)
    plt.xlabel('Date', fontsize=14)
    plt.ylabel('Sales', fontsize=14)
    plt.xticks(rotation=45)
    plt.legend()
    plt.tight_layout()
    plt.show()

def plot_bar_chart(df, categorical_column, numerical_column):
    """
    Generates a bar chart for the given categorical and numerical columns.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    categorical_column (str): The name of the categorical column.
    numerical_column (str): The name of the numerical column.

    Returns:
    None
    """
    # Aggregate the data by the categorical column
    aggregated_data = df.groupby(categorical_column)[numerical_column].sum().reset_index()

    # Generate the bar chart
    plt.figure(figsize=(12, 6))
    plt.bar(aggregated_data[categorical_column], aggregated_data[numerical_column], color='skyblue')
    plt.xlabel(categorical_column, fontsize=14)
    plt.ylabel(numerical_column, fontsize=14)
    plt.title(f'{numerical_column} by {categorical_column}', fontsize=16)
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# Example usage
# plot_sales_trend(cleaned_df, 'transaction_date', 'Total_Bill')
# plot_bar_chart(cleaned_df, 'Category', 'Sales')

## 4. Statistical Analysis
This section includes:
- A/B testing to evaluate the effectiveness of pricing strategies.
- Correlation analysis between product prices and sales volume.

Statistical Analysis

In [ ]:
from scipy.stats import ttest_ind

def ab_test(group1, group2):
    """
    Performs a two-sample t-test to compare the means of two groups.

    Parameters:
    group1 (pd.Series): The first group of data.
    group2 (pd.Series): The second group of data.

    Returns:
    tuple: A tuple containing the t-statistic and p-value.
    """
    # Check if inputs are numeric
    if not (pd.api.types.is_numeric_dtype(group1) and pd.api.types.is_numeric_dtype(group2)):
        raise ValueError("Both inputs must be numeric pandas Series.")

    # Perform the two-sample t-test
    t_stat, p_value = ttest_ind(group1, group2, nan_policy='omit')

    return t_stat, p_value

def correlation_analysis(df, price_column, quantity_column):
    """
    Calculates the correlation between product prices and sales quantities in a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame containing the data.
    price_column (str): The name of the column containing product prices.
    quantity_column (str): The name of the column containing sales quantities.

    Returns:
    float: The correlation coefficient between product prices and sales quantities, or None if an error occurs.
    """
    try:
        # Check if the specified columns exist in the DataFrame
        if price_column not in df.columns or quantity_column not in df.columns:
            raise ValueError(f"Columns '{price_column}' and/or '{quantity_column}' not found in the DataFrame.")

        # Check if the columns are numeric
        if not (pd.api.types.is_numeric_dtype(df[price_column]) and pd.api.types.is_numeric_dtype(df[quantity_column])):
            raise ValueError(f"Columns '{price_column}' and '{quantity_column}' must be numeric.")

        # Drop rows with missing values in the specified columns
        df_cleaned = df.dropna(subset=[price_column, quantity_column])

        # Calculate the correlation coefficient
        correlation = df_cleaned[price_column].corr(df_cleaned[quantity_column])

        return correlation

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example usage
# t_stat, p_value = ab_test(group1, group2)
# print(f"T-statistic: {t_stat}, P-value: {p_value}")
# correlation = correlation_analysis(df, 'Price', 'Quantity')
# print(f"Correlation coefficient: {correlation}")